In [14]:
#importing dependencies
import pandas as pd
from data_handlers import *
import tensorflow as tf

In [15]:
#loading a dataset
dataset = pd.read_csv('mushrooms.csv')

In [16]:
#encoding the dataset by changing its string values to numerical values by using a function from data_handlers.py
encoded_data = prepare_dataset(dataset)

In [17]:
#splitting the dataset to labels and features, using a function from data_handlers.py
X, y =  labels_vs_features(encoded_data)

In [18]:
#splitting the dataset to the training set and the testing set
X_train, X_test, y_train, y_test =  data_splitter(X, y)

X_train size --> (6499, 22)
y_train size --> (6499,)
X_test size --> (1625, 22)
y_test size --> (1625,)


### Defining Tensorflow NN

In [19]:
#Tensorflow placeholders - inputs to the TF graph
inputs =  tf.placeholder(tf.float32, [None, 22], name='Inputs')
targets =  tf.placeholder(tf.float32, [None, 1], name='Targets')

In [20]:
#Helper functions to define weights and biases

def init_weights(shape):
    '''
    Input: shape -  this is the shape of a matrix used to represent weigts for the arbitrary layer
    Output: wights randomly generated with size = shape
    '''
    return tf.Variable(tf.truncated_normal(shape, 0.05, 0.05))

def init_biases(shape):
    '''
    Input: shape -  this is the shape of a vector used to represent biases for the arbitrary layer
    Output: a vector for biases (all zeros) lenght = shape
    '''
    return tf.Variable(tf.zeros(shape))

In [21]:
def fully_connected_layer(inputs, input_shape, output_shape, activation=tf.nn.sigmoid):
    '''
    This function is used to create tensorflow fully connected layer.
    
    Inputs: inputs - input data to the layer
            input_shape - shape of the inputs features (number of nodes from the previous layer)
            output_shape - shape of the layer
            activatin - used as an activation function for the layer (non-liniarity)
    Output: layer - tensorflow fully connected layer
    
    '''
    #definine weights and biases
    weights = init_weights([input_shape, output_shape])
    biases = init_biases([output_shape])
    
    #x*W + b <- computation for the layer values
    layer = tf.matmul(inputs, weights) + biases
    
    #if activation argument is not None, we put layer values through an activation function
    if activation != None:
        layer = activation(layer)
        
    return layer

In [22]:
#defining the network
l1 = fully_connected_layer(inputs, 22, 50)
l2 = fully_connected_layer(l1, 50, 25)
l3 = fully_connected_layer(l2, 25, 1, activation=None)

In [23]:
#defining special parameter for our predictions - later used for testing
predictions = tf.nn.sigmoid(l3)

In [24]:
#Mean_squared_error function and optimizer choice - Classical Gradient Descent
cost = loss2 = tf.reduce_mean(tf.squared_difference(targets, predictions))
learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [25]:
epochs = 100000
batch_size = 50
from tqdm import tqdm

In [26]:
#Starting session for the graph
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    #TRAINING PORTION OF THE SESSION
    for i in tqdm(range(epochs)):
        idx = np.random.choice(len(X_train), batch_size, replace=True)
        X_batch = X_train[idx, :]
        y_batch = y_train[idx]
        y_batch = np.reshape(y_batch, (len(y_batch), 1))
        
        batch_loss, opt = sess.run([cost, optimizer], feed_dict={inputs:X_batch, targets:y_batch})
        
        if i % 10000 == 0:
            print(batch_loss)
    
    #TESTING PORTION OF THE SESSION
    preds = sess.run([predictions], feed_dict={inputs:X_test})
    true_preds = []
    for i in range(len(preds)):
        if preds[0][i] >= 0.5:
            true_preds.append(1)
        else:
            true_preds.append(0)
    
    true_correct = 0
    for i in range(len(preds)):
        if true_preds[i] == y_test[i]:
            true_correct += 1
    
    print("Accuracy: ", true_correct/len(true_preds))
        
    

  0%|                                                                                       | 0/100000 [00:00<?, ?it/s]

0.375959
0.375959


 10%|███████▍                                                                   | 9976/100000 [00:12<01:48, 830.97it/s]

0.249297
0.249297


 20%|██████████████▊                                                           | 19989/100000 [00:23<01:25, 933.72it/s]

...

0.077396
0.077396


 90%|██████████████████████████████████████████████████████████████████▌       | 89982/100000 [01:46<00:11, 908.89it/s]

0.0683184
0.0683184


100%|█████████████████████████████████████████████████████████████████████████| 100000/100000 [01:58<00:00, 843.91it/s]


Accuracy:  1.0
Accuracy:  1.0
